In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
import activity_image_dataset as aids
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import dcnn

In [2]:
train_signals_dir = []
label_names = {
  0: 'WALKING',
  1: 'WALKING_UPSTAIRS',
  2: 'WALKING_DOWNSTAIRS',
  3: 'SITTING',
  4: 'STANDING',
  5: 'LAYING'
}
signal_names = ['body_gyro_x', 'body_gyro_y', 'body_gyro_z', 'total_acc_x', 'total_acc_y', 'total_acc_z', 'body_acc_x', 'body_acc_y', 'body_acc_z']
for name in signal_names:
  path = 'C:/Users/Cesar/Documents/projects/har/datasets/UCI HAR Dataset/train/Inertial Signals/{}_train.txt'.format(name)
  train_signals_dir.append(path)
  
train_label_dir = 'C:/Users/Cesar/Documents/projects/har/datasets/UCI HAR Dataset/train/y_train.txt'

print(train_signals_dir)
print(train_label_dir)

['C:/Users/Cesar/Documents/projects/har/datasets/UCI HAR Dataset/train/Inertial Signals/body_gyro_x_train.txt', 'C:/Users/Cesar/Documents/projects/har/datasets/UCI HAR Dataset/train/Inertial Signals/body_gyro_y_train.txt', 'C:/Users/Cesar/Documents/projects/har/datasets/UCI HAR Dataset/train/Inertial Signals/body_gyro_z_train.txt', 'C:/Users/Cesar/Documents/projects/har/datasets/UCI HAR Dataset/train/Inertial Signals/total_acc_x_train.txt', 'C:/Users/Cesar/Documents/projects/har/datasets/UCI HAR Dataset/train/Inertial Signals/total_acc_y_train.txt', 'C:/Users/Cesar/Documents/projects/har/datasets/UCI HAR Dataset/train/Inertial Signals/total_acc_z_train.txt', 'C:/Users/Cesar/Documents/projects/har/datasets/UCI HAR Dataset/train/Inertial Signals/body_acc_x_train.txt', 'C:/Users/Cesar/Documents/projects/har/datasets/UCI HAR Dataset/train/Inertial Signals/body_acc_y_train.txt', 'C:/Users/Cesar/Documents/projects/har/datasets/UCI HAR Dataset/train/Inertial Signals/body_acc_z_train.txt']
C:/

In [3]:
uci_dataset = aids.ActivityImageDataset(train_signals_dir, train_label_dir)
train_dataloader = DataLoader(uci_dataset, batch_size=32, shuffle=True)
print(len(train_dataloader.dataset))

c:\Users\Cesar\Documents\projects\har\har-dcnn\activity_image_dataset.py:11: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  signal_data = pd.read_csv(signal_dir, delim_whitespace=True, header=None).values
c:\Users\Cesar\Documents\projects\har\har-dcnn\activity_image_dataset.py:11: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  signal_data = pd.read_csv(signal_dir, delim_whitespace=True, header=None).values
c:\Users\Cesar\Documents\projects\har\har-dcnn\activity_image_dataset.py:11: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  signal_data = pd.read_csv(signal_dir, delim_whitespace=True, header=None).values
c:\Users\Cesar\Documents\projects\har\har-dcnn\activity_image_dataset.py:11: FutureWarning: Th

7352


c:\Users\Cesar\Documents\projects\har\har-dcnn\activity_image_dataset.py:15: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  self.labels = pd.read_csv(label_dir, delim_whitespace=True, header=None).values


In [4]:
images, labels = next(iter(train_dataloader))
labels = labels.flatten()

fig = make_subplots(rows=8, cols=4)
for i, img in enumerate(images): 
  row = i // 4
  col = i % 4
  img = img[0]
  fig.add_trace(px.imshow(np.log(img + 1), color_continuous_scale='gray').data[0], row=row+1, col=col+1)
  fig.update_xaxes(title_text=f'Label: {label_names[int(labels[i])]}', row=row+1, col=col+1)
  fig.update_yaxes(title_text=f'Image {i+1}', row=row+1, col=col+1)

fig.update_layout(
  {
    "autosize": True,
    "margin": dict(l=10, r=10, t=10, b=10),
    "template": "plotly_white",
    "height": 1200  
  }
)
fig.show()

c:\Users\Cesar\Documents\projects\har\har-dcnn\activity_image_dataset.py:23: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:277.)
  activity_image = torch.tensor([activity_image[:36]], dtype=torch.float32)


In [5]:
import torch.nn.functional as F

def train(train_loader, model, criterion, optimizer, num_epochs):
  for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch, num_epochs - 1))
    print('-' * 10)
    model.train()
    running_loss = 0.0
    running_corrects = 0
    for inputs, labels in train_loader:
      optimizer.zero_grad()
      outputs = model(inputs)
      _,preds = torch.max(outputs,1)
      #labels_one_hot = F.one_hot(labels, 6).float()
      loss = criterion(outputs, labels)      
      loss.backward()
      optimizer.step()
      #statistics
      running_loss += loss.item() * inputs.size(0)
      running_corrects += torch.sum(preds == labels.data)
    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_acc = running_corrects.double() / len(train_loader.dataset)
    print('Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))
  print('Finished training')
  

In [6]:
num_classes = 6
model = dcnn.DCNN(num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# train model
train(train_dataloader, model, criterion, optimizer, 50)

Epoch 0/49
----------
Loss: 1.5048 Acc: 0.5521
Epoch 1/49
----------
Loss: 1.3332 Acc: 0.7163
Epoch 2/49
----------
Loss: 1.3086 Acc: 0.7376
Epoch 3/49
----------
Loss: 1.2019 Acc: 0.8478
Epoch 4/49
----------
Loss: 1.1450 Acc: 0.9034
Epoch 5/49
----------
Loss: 1.1334 Acc: 0.9140
Epoch 6/49
----------
Loss: 1.1298 Acc: 0.9153
Epoch 7/49
----------
Loss: 1.1251 Acc: 0.9204
Epoch 8/49
----------
Loss: 1.1190 Acc: 0.9248
Epoch 9/49
----------
Loss: 1.1168 Acc: 0.9291
Epoch 10/49
----------
Loss: 1.1081 Acc: 0.9374
Epoch 11/49
----------
Loss: 1.1030 Acc: 0.9434
Epoch 12/49
----------
Loss: 1.0985 Acc: 0.9468
Epoch 13/49
----------
Loss: 1.0945 Acc: 0.9508
Epoch 14/49
----------
Loss: 1.0974 Acc: 0.9472
Epoch 15/49
----------
Loss: 1.0956 Acc: 0.9493
Epoch 16/49
----------
Loss: 1.0931 Acc: 0.9517
Epoch 17/49
----------
Loss: 1.0923 Acc: 0.9520
Epoch 18/49
----------
Loss: 1.0976 Acc: 0.9468
Epoch 19/49
----------
Loss: 1.0899 Acc: 0.9540
Epoch 20/49
----------
Loss: 1.0932 Acc: 0.9517
Ep

## Test

In [7]:
signal_names = ['body_gyro_x', 'body_gyro_y', 'body_gyro_z', 'total_acc_x', 'total_acc_y', 'total_acc_z', 'body_acc_x', 'body_acc_y', 'body_acc_z']
test_signals_dir = []
for name in signal_names:
  path = 'C:/Users/Cesar/Documents/projects/har/datasets/UCI HAR Dataset/test/Inertial Signals/{}_test.txt'.format(name)
  test_signals_dir.append(path)
  
test_label_dir = 'C:/Users/Cesar/Documents/projects/har/datasets/UCI HAR Dataset/test/y_test.txt'

test_uci_dataset = aids.ActivityImageDataset(test_signals_dir, test_label_dir)
test_loader = DataLoader(test_uci_dataset, batch_size=32, shuffle=True)


c:\Users\Cesar\Documents\projects\har\har-dcnn\activity_image_dataset.py:11: FutureWarning:

The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead

c:\Users\Cesar\Documents\projects\har\har-dcnn\activity_image_dataset.py:11: FutureWarning:

The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead

c:\Users\Cesar\Documents\projects\har\har-dcnn\activity_image_dataset.py:11: FutureWarning:

The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead

c:\Users\Cesar\Documents\projects\har\har-dcnn\activity_image_dataset.py:11: FutureWarning:

The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead

c:\Users\Cesar\Documents\projects\har\har-dcnn\activity_image_dataset.py:11: FutureWarning:

The 'delim_whitespace' 

In [8]:

# Evaluación del modelo
model.eval()
correct = 0
total = 0
with torch.no_grad():
  for inputs, labels in test_loader:
    outputs = model(inputs)
    print(labels)
    _, predicted = torch.max(outputs, 1)
    print(predicted)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total}%')

tensor([2, 1, 2, 4, 1, 4, 3, 5, 1, 4, 4, 0, 1, 1, 2, 4, 1, 3, 0, 3, 2, 0, 5, 5,
        4, 3, 5, 5, 2, 4, 4, 3])
tensor([2, 1, 2, 4, 1, 4, 3, 5, 1, 4, 4, 0, 1, 1, 2, 4, 1, 3, 0, 3, 2, 0, 5, 5,
        4, 3, 5, 5, 2, 4, 4, 3])
tensor([5, 5, 1, 0, 2, 4, 0, 4, 4, 5, 1, 0, 1, 2, 0, 4, 1, 5, 2, 2, 3, 0, 5, 5,
        3, 3, 0, 0, 0, 4, 0, 0])
tensor([5, 5, 1, 0, 2, 4, 0, 4, 3, 5, 1, 0, 1, 1, 0, 4, 1, 5, 2, 2, 3, 0, 5, 5,
        3, 4, 0, 0, 0, 4, 0, 0])
tensor([3, 5, 2, 5, 5, 4, 3, 5, 4, 4, 4, 2, 1, 5, 5, 2, 1, 0, 2, 0, 0, 3, 1, 1,
        0, 3, 4, 0, 4, 5, 4, 4])
tensor([3, 5, 2, 5, 5, 4, 3, 5, 4, 4, 4, 2, 1, 5, 5, 2, 1, 0, 2, 0, 0, 3, 1, 1,
        0, 3, 4, 0, 3, 5, 4, 4])
tensor([5, 3, 3, 0, 3, 1, 4, 2, 5, 1, 1, 0, 5, 4, 1, 0, 4, 0, 0, 1, 5, 4, 2, 1,
        3, 3, 1, 0, 2, 3, 3, 3])
tensor([5, 3, 4, 0, 4, 1, 4, 2, 5, 1, 1, 1, 5, 4, 2, 0, 4, 0, 0, 1, 5, 4, 2, 1,
        3, 3, 1, 0, 2, 3, 4, 3])
tensor([1, 2, 3, 4, 0, 2, 0, 2, 1, 0, 1, 5, 5, 0, 3, 4, 2, 2, 2, 2, 4, 1, 0, 4,
        5, 3, 2,